# Elecciones al Congreso de los Diputados 2019

En otros notebooks ya se ha analizado las deficiencias de ley electoral que rige en España: [Ley Orgánica 5/1985, de 19 de Junio, del régimen electoral general](http://www.juntaelectoralcentral.es/cs/jec/loreg) (LOREG). En éste se va a reiterar una vez más su principal problema: la desproporcionalidad de un sistema supuestamente proporcional.

## Accesso a los datos

Los datos utilizados son los oficiales suministrados por el Ministerio del Interior. Se utilizan los documentos <code>json</code> que usa la [página web oficial](https://www.resultados.eleccionesgenerales19.es/Congreso/Total-nacional/0/es)

El procesamiento de los datos se realizará mediante el paquete Python [`pandas`](https://pandas.pydata.org/pandas-docs/stable/index.html), en particular mediante la clase [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame), diseñada para gestionar datos en formato tabular. La estructura del `DataFrame` que se utilzará como base es la siguiente:

- Índice doble:
    * `Constituency`. Valores presentes: circunscripciones electorales, es decir, las provicinas de España.
    * `Option`. Valores presentes: `Abstención`, `Votos nulos`, `Votos en blanco`, y siglas utilizadas por los partidos.
- Columnas:
    * `Votes`: número total de votos correspondiente a una provincia y a una opción política.

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint
import urllib
import json
import unicodecsv as csv
from voting.utils import get_constituency_votes
from voting.apportionment import calculate_parliament, assign_constituency_representatives
from voting.constants import *

In [3]:
# Downloading data by constituency (circunscripcion electoral)
def download_data_from_webpage(file_name):
    url = "https://www.resultados.eleccionesgenerales19.es/assets/nomenclator.json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    levels = data["constantes"]["level"]
    parties = data["partidos"]["co"]["act"]
    synonyms = {"PSC": "PSOE", "PSdeG-PSOE": "PSOE", "PSE-EE (PSOE)": "PSOE",
                "PODEMOS-IU LV CA-EQUO": "PODEMOS-IU-EQUO",
                "PODEMOS-EUPV": "PODEMOS-IU-EQUO",
                "PODEMOS-IU-EQUO BERDEAK": "PODEMOS-IU-EQUO",
                u"PODEMOS-EU-MAREAS EN COMÚN-EQUO": "PODEMOS-IU-EQUO",
                "PODEMOS-EUIB": "PODEMOS-IU-EQUO",
                "PODEMOS-IU-EQUO-BATZARRE": "PODEMOS-IU-EQUO",
                "PODEMOS-IX-EQUO": "PODEMOS-IU-EQUO",
                "PODEMOS-IU-EQUO-AAeC": "PODEMOS-IU-EQUO",
                "PP-FORO": "PP"}

    party_names = {party["codpar"]: party["siglas"] for party in parties}
    party_names = {k: v if v not in synonyms else synonyms[v] for k, v in party_names.items()}
    regions = [place for place in data["ambitos"]["co"] if place["l"] == 2]
    region_names = {region["c"][0:2]: region["n"] for region in regions}
    places = [place for place in data["ambitos"]["co"] if place["l"] == 8]
    district_names = {place["c"]: {'name': place["n"], 'region': region_names[place["c"][0:2]]} for place in places}

    url = "https://www.resultados.eleccionesgenerales19.es/json/CO/RCO99999999999NACI.json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    districts = data["ambs"]
    votes = []
    for district in districts:
        code = district["amb"]
        name = district_names[code]["name"]
        region = district_names[code]["region"]
        seats = district["totales"]["act"]["carg"]
        universe = district["totales"]["act"]["centota"]
        habitants = district["totales"]["act"]["padron"]
        vote_parties = district["totales"]["act"]["votcan"]
        no_votes = district["totales"]["act"]["absten"]
        vote_blank = district["totales"]["act"]["votbla"]
        vote_invalid = district["totales"]["act"]["votnul"]

        votes_dict = {REGION: region, CONSTITUENCY: name, OPTION: OPTION_ABSTENTION,
                      VOTES: no_votes, SEATS: 0}
        votes.append(votes_dict)
        votes_dict = {REGION: region, CONSTITUENCY: name, OPTION: OPTION_BLANK_VOTE,
                      VOTES: vote_blank, SEATS: 0}
        votes.append(votes_dict)
        votes_dict = {REGION: region, CONSTITUENCY: name, OPTION: OPTION_INVALID_VOTE,
                      VOTES: vote_invalid, SEATS: 0}
        votes.append(votes_dict)
        for party in district["partotabla"]:
            if party["act"]["codpar"] == "0000" and int(party["act"]["vot"]) == 0:
                continue
            votes_dict = {REGION: region, CONSTITUENCY: name,
                          OPTION: party_names[party["act"]["codpar"]],
                          VOTES: int(party["act"]["vot"]), "Seats": int(party["act"]["carg"])}
            votes.append(votes_dict)

    keys = [REGION, CONSTITUENCY, OPTION, VOTES, SEATS]
    with open('./data/%s' % file_name, 'wb') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(votes)

In [19]:
download_data_from_webpage('legislative_election_2019_04.csv')
!ls ./data/legislative_election_2019*

./data/legislative_election_2019_04.csv


In [31]:
# Loading data file into a pandas.Dataframe object
df = pd.read_csv('./data/legislative_election_2019_04.csv')
dataframe = df[[CONSTITUENCY, OPTION, VOTES]]
dataframe.set_index([CONSTITUENCY, OPTION], inplace=True)
df.tail()

,Region,Constituency,Option,Votes,Seats
733,Aragón,Zaragoza,PUM+J,734,0
734,Aragón,Zaragoza,FIA,650,0
735,Aragón,Zaragoza,PYLN,611,0
736,Aragón,Zaragoza,PCTE,582,0
737,Aragón,Zaragoza,PCPE,452,0


In [32]:
# Some descriptive info about the dataframe
print("Column names:                       %s" % ', '.join(list(df.columns)))
print("Data frame shape (#rows, #columns): %s" % str(df.shape))
print("Different values in opcion column:  %s" % ', '.join(df[OPTION].unique()))

Column names:                       Region, Constituency, Option, Votes, Seats
Data frame shape (#rows, #columns): (738, 5)
Different values in opcion column:  Abstención, Votos en blanco, Votos nulos, PSOE, PP, PODEMOS-IU-EQUO, Cs, BNG, VOX, PACMA, EN MAREA, PCOE, RECORTES CERO-GV, CxG, PCTG, RECORTES CERO-GV-PCAS-TC, PUM+J, PCPE, PCTE, COMPROMÍS 2019, AVANT-LOS VERDES, ERPV, P-LIB, EB, PCPA, EAJ-PNV, EH Bildu, PCTE/ELAK, PACT, ANDECHA ASTUR, PH, FE de las JONS, C.Ex-C.R.Ex-P.R.Ex, DP, ERC-SOBIRANISTES, ECP-GUANYEM EL CANVI, JxCAT-JUNTS, FRONT REPUBLICÀ, PCPC, IZQP, PCTC, CNV, SOLIDARIA, AxSI, PRC, AVANT ADELANTE LOS VERDES, PDSJE-UDEC, ELAK/PCTE, "JF", IZAR, FIA, PYLN, ARA-MES-ESQUERRA, EL PI, CILU-LINARES, RISA, PR+, NCa, CCa-PNC, AHORA CANARIAS, PREPAL, C 21, CpM, SOMOS REGIÓN, DPL, NA+, GBAI, VOU, F8, centrados, PPSO, +MAS+, UDT, UIG-SOM-CUIDES, UNIÓN REGIONALISTA


In [33]:
# Adding votes globally (Spain as a whole)
spain_df = dataframe.groupby([OPTION]).sum().reset_index()

In [34]:
# Constituencies and their seats
constituencies = df[[CONSTITUENCY, SEATS]].groupby(by=CONSTITUENCY).agg({SEATS: sum}).to_dict()[SEATS]
seats = sum(constituencies.values())
print("Total seats in Spain: %d" % seats)

Total seats in Spain: 350


## Funciones

In [35]:
# Sum the votes from the parties in every coalition, 
# supress and append rows in the dataframe accordingly
def process_coalition(df, coalitions):
    dataframe = df[:]
    for coalition in coalitions:
        coalition_result = dataframe[dataframe[OPTION].isin(coalition)]
        coalition_df = coalition_result[[CONSTITUENCY, VOTES]].groupby(by=CONSTITUENCY).agg({VOTES: sum})
        coalition_df[OPTION] = " & ".join(coalition)
        coalition_df.reset_index(inplace=True)
        coalition_df = coalition_df[[CONSTITUENCY, OPTION, VOTES]]
        dataframe = dataframe[~dataframe[OPTION].isin(coalition)]
        dataframe = dataframe.append(coalition_df)
    dataframe.set_index([CONSTITUENCY, OPTION], inplace=True)
    return dataframe

## Reparto según la legislación vigente (ley D'Hondt)

Aquí se realiza el cálculo del reparto de diputados de las elecciones al Congreso de los Diputados de 2019. Debe coincidir con lo que se muestra en la [página oficinal](https://www.resultados.eleccionesgenerales19.es/Congreso/Total-nacional/0/es)

In [36]:
parliament = calculate_parliament(dataframe, constituencies, formula="d'Hondt", verbose=False)
seats = parliament["Seats"].sum()
print("Total seats: %d\n\n" % seats)
parliament

Total seats: 350




,Votes,Seats
Party,,
PSOE,7480755,123
PP,4356023,66
Cs,4136600,57
PODEMOS-IU-EQUO,3118191,35
VOX,2677173,24
ERC-SOBIRANISTES,1015355,15
ECP-GUANYEM EL CANVI,614738,7
JxCAT-JUNTS,497638,7
EAJ-PNV,394627,6


## Reparto según una circunscripción única con la ley D'Hondt

Se suele decir que el sistema electoral español es proporcional en teoría, pero mayoritario en la práctica. Aunque es un lugar común decir que los grandes beneficiados son los partidos nacionalistas (convertidos ahora en claramente independentistas), en realidad son los partidos mayoritarios los que han obtenido un número desproporcionado de diputados en las zonas menos pobladas de España.

La forma más simple de conseguir un sistema más proporcional es convertir todo el país en una única circunscripción. Si además se mantiene el umbral mínimo del 3% del voto válido para participar en la asignación de escaños, el resultado habría sido:

In [37]:
parliament = assign_constituency_representatives(spain_df, seats, minimum_percentage=3.0)
parliament

,Votes,Seats
Option,,
PSOE,7480755,115
PP,4356023,67
Cs,4136600,64
PODEMOS-IU-EQUO,3118191,48
VOX,2677173,41
ERC-SOBIRANISTES,1015355,15


## Reparto sin exigir un porcentaje mínimo de voto

En el apartado anterior se ve claramente que pocos partidos llegan al 3% del voto válido. Por tanto, ese sistema seguiría sin ser completamente proporcional. Si se elimina el requisito del 3%, el resultado sería:

In [38]:
parliament = assign_constituency_representatives(spain_df, seats, minimum_percentage=0.0)
parliament

,Votes,Seats
Option,,
PSOE,7480755,105
PP,4356023,61
Cs,4136600,58
PODEMOS-IU-EQUO,3118191,43
VOX,2677173,37
ERC-SOBIRANISTES,1015355,14
ECP-GUANYEM EL CANVI,614738,8
JxCAT-JUNTS,497638,6
EAJ-PNV,394627,5


Con la ley electoral actual <i>PACMA</i>, con 70.000 votos más que <i>EH Bildu</i>, no ha conseguido ningún escaño, mientras que los independentistas vascos han obtenido 4 diputados. En una circunscripción única sin porcentaje mínimo de voto, <i>PACMA</i> habría obtenido 4 y <i>EH Bildu</i> 3.

## Y si PP y VOX se hubiesen presentados juntos

En la campaña electoral ha quedado claro que las diferencias programáticas entre PP y VOX eran menores de las que se pensaban en un principio. ¿Qué hubiera ocurrido si hubiesen formado una coalición?

In [43]:
coalitions = np.array([['PP', 'VOX']])
new_df = process_coalition(df[[CONSTITUENCY, OPTION, VOTES]], coalitions)
parliament = calculate_parliament(new_df, constituencies, formula="d'Hondt", verbose=False)
parliament

,Votes,Seats
Party,,
PP & VOX,7033196,112
PSOE,7480755,112
Cs,4136600,54
PODEMOS-IU-EQUO,3118191,30
ERC-SOBIRANISTES,1015355,15
JxCAT-JUNTS,497638,7
EAJ-PNV,394627,6
ECP-GUANYEM EL CANVI,614738,6
EH Bildu,258840,3


### Circunscripción única y porcentaje mínimo de voto

In [44]:
spain_df = new_df.groupby([OPTION]).sum().reset_index()
parliament = assign_constituency_representatives(spain_df, seats, minimum_percentage=3.0)
parliament

,Votes,Seats
Option,,
PSOE,7480755,115
PP & VOX,7033196,108
Cs,4136600,64
PODEMOS-IU-EQUO,3118191,48
ERC-SOBIRANISTES,1015355,15


### Circunscripción única y sin porcentaje mínimo de voto

In [45]:
parliament = assign_constituency_representatives(spain_df, seats, minimum_percentage=0.0)
parliament

,Votes,Seats
Option,,
PSOE,7480755,105
PP & VOX,7033196,98
Cs,4136600,58
PODEMOS-IU-EQUO,3118191,43
ERC-SOBIRANISTES,1015355,14
ECP-GUANYEM EL CANVI,614738,8
JxCAT-JUNTS,497638,6
EAJ-PNV,394627,5
PACMA,326045,4


### Circunscripción única, sin porcentaje mínimo de voto y otras coaliciones previsibles

Es previsible que ante una circunscripción única (como ya ocurre en las elecciones europeas) los partidos políticos formen múltiples coaliciones para conseguir la mayor representanción posible. Uniendo todas las marcas de <i>Podemos</i> por un lado, y todo el independentismo catalán por otro lado, el resultado habría sido:

In [48]:
coalitions = np.array([['PP', 'VOX'], 
                       ['PODEMOS-IU-EQUO', 'ECP-GUANYEM EL CANVI', 'COMPROMÍS 2019'], 
                       ['ERC-SOBIRANISTES', 'JxCAT-JUNTS', 'FRONT REPUBLICÀ']])
new_df = process_coalition(df[[CONSTITUENCY, OPTION, VOTES]], coalitions)
spain_df = new_df.groupby([OPTION]).sum().reset_index()
parliament = assign_constituency_representatives(spain_df, seats, minimum_percentage=0.0)
parliament

,Votes,Seats
Option,,
PSOE,7480755,104
PP & VOX,7033196,98
Cs,4136600,57
PODEMOS-IU-EQUO & ECP-GUANYEM EL CANVI & COMPROMÍS 2019,3905680,54
ERC-SOBIRANISTES & JxCAT-JUNTS & FRONT REPUBLICÀ,1626001,22
EAJ-PNV,394627,5
PACMA,326045,4
EH Bildu,258840,3
BNG,93810,1


## Conclusiones

Ahora que el parlamento español está fragmentado en cinco partidos nacionales que superan el 10% del voto emitido, es el momento para reformar de una vez la ley electoral. La circunscripción única sin porcentaje mínimo de voto es una opción razonable: aseguraría una representación proporcional; no habría grandes perjudicados (quizás el PSOE); el nacionalismo conseguiría prácticamente la misma representación; y fuerzas (como el <i>PACMA</i>) con el voto muy disperso conseguirían estar representadas.